In [20]:
import os 
import pandas as pd

os.chdir("../")



In [11]:
%pwd

'd:\\Projects'

In [12]:
data = pd.read_csv("D:/Projects/dsproject/artifacts/data_ingestion/winequality-red.csv")

In [13]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [15]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [21]:
from dataclasses import dataclass
from pathlib import Path



@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

    

In [22]:
from src.datascience.entity.config_entity import DataIngestionConfig
from src.datascience.utils.common import read_yaml,create_directories
from src.datascience.constants import *

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        

        create_directories([config.root_dir])
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config




In [23]:
from src.datascience import logger
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    break
                else:
                    validation_status = True
            
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

    

In [24]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'

In [6]:
# Check CSV columns vs Schema columns
import yaml

csv_columns = list(data.columns)
print("CSV Columns:")
for i, col in enumerate(csv_columns):
    print(f"{i}: '{col}'")

schema = yaml.safe_load(open('schema.yaml'))
schema_columns = list(schema['COLUMNS'].keys())
print("\nSchema Columns:")
for i, col in enumerate(schema_columns):
    print(f"{i}: '{col}'")

print("\nMissing in Schema:")
for col in csv_columns:
    if col not in schema_columns:
        print(f"  '{col}'")

print("\nMissing in CSV:")
for col in schema_columns:
    if col not in csv_columns:
        print(f"  '{col}'")

CSV Columns:
0: 'fixed acidity'
1: 'volatile acidity'
2: 'citric acid'
3: 'residual sugar'
4: 'chlorides'
5: 'free sulfur dioxide'
6: 'total sulfur dioxide'
7: 'density'
8: 'pH'
9: 'sulphates'
10: 'alcohol'
11: 'quality'

Schema Columns:
0: 'fixed acidity'
1: 'volatile acidity'
2: 'citric acid'
3: 'residual sugar'
4: 'chlorides'
5: 'free sulfur dioxide'
6: 'total sulfur dioxide'
7: 'density'
8: 'pH'
9: 'sulphates'
10: 'alcohol'
11: 'quality'

Missing in Schema:

Missing in CSV:


In [7]:
# Debug the validation logic
validation_status = None
all_cols = list(data.columns)
all_schema = schema['COLUMNS'].keys()

print("Starting validation...")
print(f"Initial validation_status: {validation_status}")

for i, col in enumerate(all_cols):
    print(f"\nIteration {i}: Checking column '{col}'")
    if col not in all_schema:
        print(f"  -> Column NOT in schema! Setting validation_status = False")
        validation_status = False
        # In the old code, it would write to file here
    else:
        print(f"  -> Column IS in schema! Setting validation_status = True")
        validation_status = True
        # In the old code, it would write to file here (OVERWRITING previous value!)
    print(f"  -> Current validation_status: {validation_status}")

print(f"\nFinal validation_status: {validation_status}")

Starting validation...
Initial validation_status: None

Iteration 0: Checking column 'fixed acidity'
  -> Column IS in schema! Setting validation_status = True
  -> Current validation_status: True

Iteration 1: Checking column 'volatile acidity'
  -> Column IS in schema! Setting validation_status = True
  -> Current validation_status: True

Iteration 2: Checking column 'citric acid'
  -> Column IS in schema! Setting validation_status = True
  -> Current validation_status: True

Iteration 3: Checking column 'residual sugar'
  -> Column IS in schema! Setting validation_status = True
  -> Current validation_status: True

Iteration 4: Checking column 'chlorides'
  -> Column IS in schema! Setting validation_status = True
  -> Current validation_status: True

Iteration 5: Checking column 'free sulfur dioxide'
  -> Column IS in schema! Setting validation_status = True
  -> Current validation_status: True

Iteration 6: Checking column 'total sulfur dioxide'
  -> Column IS in schema! Setting va